<a href="https://colab.research.google.com/github/kjedrzejczak/Wprowadzenie_do_sieci_neuronowych-Tensorflow_2.0-Keras/blob/main/ANN_regresja_20.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

np.set_printoptions(precision=12, suppress=True, linewidth=150)
pd.options.display.float_format = '{:.6f}'.format

In [2]:
raw_dataset = pd.read_csv('https://storage.googleapis.com/esmartdata-courses-files/ann-course/housing.csv')
raw_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20640 entries, 0 to 20639
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   longitude           20640 non-null  float64
 1   latitude            20640 non-null  float64
 2   housing_median_age  20640 non-null  float64
 3   total_rooms         20640 non-null  float64
 4   total_bedrooms      20433 non-null  float64
 5   population          20640 non-null  float64
 6   households          20640 non-null  float64
 7   median_income       20640 non-null  float64
 8   median_house_value  20640 non-null  float64
 9   ocean_proximity     20640 non-null  object 
dtypes: float64(9), object(1)
memory usage: 1.6+ MB


In [3]:
dataset = raw_dataset.copy()
dataset.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.230000,37.880000,41.000000,880.000000,129.000000,322.000000,126.000000,8.325200,452600.000000,NEAR BAY
1,-122.220000,37.860000,21.000000,7099.000000,1106.000000,2401.000000,1138.000000,8.301400,358500.000000,NEAR BAY
2,-122.240000,37.850000,52.000000,1467.000000,190.000000,496.000000,177.000000,7.257400,352100.000000,NEAR BAY
3,-122.250000,37.850000,52.000000,1274.000000,235.000000,558.000000,219.000000,5.643100,341300.000000,NEAR BAY
4,-122.250000,37.850000,52.000000,1627.000000,280.000000,565.000000,259.000000,3.846200,342200.000000,NEAR BAY


In [4]:
dataset.isnull().sum() / len(dataset)

longitude            0.000000
latitude             0.000000
housing_median_age   0.000000
total_rooms          0.000000
total_bedrooms       0.010029
population           0.000000
households           0.000000
median_income        0.000000
median_house_value   0.000000
ocean_proximity      0.000000
dtype: float64

In [5]:
dataset = dataset.dropna()

In [6]:
dataset.isnull().sum() / len(dataset)

longitude            0.000000
latitude             0.000000
housing_median_age   0.000000
total_rooms          0.000000
total_bedrooms       0.000000
population           0.000000
households           0.000000
median_income        0.000000
median_house_value   0.000000
ocean_proximity      0.000000
dtype: float64

In [7]:
dataset.describe()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
count,20433.000000,20433.000000,20433.000000,20433.000000,20433.000000,20433.000000,20433.000000,20433.000000,20433.000000
mean,-119.570689,35.633221,28.633094,2636.504233,537.870553,1424.946949,499.433465,3.871162,206864.413155
std,2.003578,2.136348,12.591805,2185.269567,421.385070,1133.208490,382.299226,1.899291,115435.667099
min,-124.350000,32.540000,1.000000,2.000000,1.000000,3.000000,1.000000,0.499900,14999.000000
25%,-121.800000,33.930000,18.000000,1450.000000,296.000000,787.000000,280.000000,2.563700,119500.000000
50%,-118.490000,34.260000,29.000000,2127.000000,435.000000,1166.000000,409.000000,3.536500,179700.000000
75%,-118.010000,37.720000,37.000000,3143.000000,647.000000,1722.000000,604.000000,4.744000,264700.000000
max,-114.310000,41.950000,52.000000,39320.000000,6445.000000,35682.000000,6082.000000,15.000100,500001.000000


In [8]:
dataset.describe(include='object')

,ocean_proximity
count,20433
unique,5
top,<1H OCEAN
freq,9034


In [9]:
dataset.ocean_proximity.value_counts()

ocean_proximity
<1H OCEAN     9034
INLAND        6496
NEAR OCEAN    2628
NEAR BAY      2270
ISLAND           5
Name: count, dtype: int64

In [10]:
px.histogram(dataset, x='median_house_value')

In [11]:
dataset.median_house_value.value_counts()

median_house_value
500001.000000    958
137500.000000    119
162500.000000    116
112500.000000    103
187500.000000     92
                ... 
359200.000000      1
51200.000000       1
39800.000000       1
377600.000000      1
47000.000000       1
Name: count, Length: 3833, dtype: int64

In [12]:
index_to_drop = dataset[dataset.median_house_value == 500001].index
dataset = dataset.drop(index=index_to_drop)
px.histogram(dataset, x='median_house_value') # 👈👈👈👈👌👌

In [13]:
dataset_dummies = pd.get_dummies(dataset, drop_first=True, dtype='float')
dataset_dummies.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity_INLAND,ocean_proximity_ISLAND,ocean_proximity_NEAR BAY,ocean_proximity_NEAR OCEAN
0,-122.230000,37.880000,41.000000,880.000000,129.000000,322.000000,126.000000,8.325200,452600.000000,0.000000,0.000000,1.000000,0.000000
1,-122.220000,37.860000,21.000000,7099.000000,1106.000000,2401.000000,1138.000000,8.301400,358500.000000,0.000000,0.000000,1.000000,0.000000
2,-122.240000,37.850000,52.000000,1467.000000,190.000000,496.000000,177.000000,7.257400,352100.000000,0.000000,0.000000,1.000000,0.000000
3,-122.250000,37.850000,52.000000,1274.000000,235.000000,558.000000,219.000000,5.643100,341300.000000,0.000000,0.000000,1.000000,0.000000
4,-122.250000,37.850000,52.000000,1627.000000,280.000000,565.000000,259.000000,3.846200,342200.000000,0.000000,0.000000,1.000000,0.000000


# Podział na dane treningowe oraz testowe




In [14]:
train_dataset = dataset_dummies.sample(frac=0.8, random_state=0)  # 👈👈👈👈👌
test_dataset = dataset_dummies.drop(train_dataset.index)           # 👈👈👈👈👌

In [15]:
train_dataset.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity_INLAND,ocean_proximity_ISLAND,ocean_proximity_NEAR BAY,ocean_proximity_NEAR OCEAN
19234,-122.690000,38.510000,18.000000,3364.000000,501.000000,1442.000000,506.000000,6.685400,313000.000000,0.000000,0.000000,0.000000,0.000000
1859,-124.140000,41.950000,21.000000,2696.000000,578.000000,1208.000000,494.000000,2.275000,122400.000000,0.000000,0.000000,0.000000,1.000000
51,-122.270000,37.820000,43.000000,1868.000000,456.000000,1061.000000,407.000000,1.504500,93800.000000,0.000000,0.000000,1.000000,0.000000
11192,-117.930000,33.820000,28.000000,2444.000000,555.000000,1848.000000,567.000000,3.017900,198800.000000,0.000000,0.000000,0.000000,0.000000
20355,-118.960000,34.190000,16.000000,1807.000000,346.000000,587.000000,296.000000,1.981100,162500.000000,0.000000,0.000000,0.000000,0.000000


In [16]:
test_dataset.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity_INLAND,ocean_proximity_ISLAND,ocean_proximity_NEAR BAY,ocean_proximity_NEAR OCEAN
10,-122.260000,37.850000,52.000000,2202.000000,434.000000,910.000000,402.000000,3.203100,281500.000000,0.000000,0.000000,1.000000,0.000000
13,-122.260000,37.840000,52.000000,696.000000,191.000000,345.000000,174.000000,2.673600,191300.000000,0.000000,0.000000,1.000000,0.000000
19,-122.270000,37.840000,52.000000,1503.000000,298.000000,690.000000,275.000000,2.603300,162900.000000,0.000000,0.000000,1.000000,0.000000
26,-122.280000,37.850000,49.000000,1130.000000,244.000000,607.000000,239.000000,2.459700,93800.000000,0.000000,0.000000,1.000000,0.000000
40,-122.260000,37.830000,52.000000,1665.000000,419.000000,946.000000,395.000000,2.097800,155400.000000,0.000000,0.000000,1.000000,0.000000


In [17]:
print(len(train_dataset))
print(len(test_dataset))

15580
3895


In [18]:
px.scatter_matrix(train_dataset, dimensions=['median_house_value', 'housing_median_age', 'median_income', 'total_rooms'], color='median_house_value', height=700)

In [19]:
train_stats = train_dataset.describe()
train_stats.pop('median_house_value')
train_stats = train_stats.transpose()
train_stats

,count,mean,std,min,25%,50%,75%,max
longitude,15580.000000,-119.558780,2.006237,-124.350000,-121.750000,-118.490000,-117.990000,-114.470000
latitude,15580.000000,35.648614,2.147016,32.550000,33.930000,34.270000,37.730000,41.950000
housing_median_age,15580.000000,28.489217,12.505895,1.000000,18.000000,29.000000,37.000000,52.000000
total_rooms,15580.000000,2620.014506,2195.654212,2.000000,1441.000000,2112.000000,3119.000000,39320.000000
total_bedrooms,15580.000000,539.900578,424.499548,2.000000,299.000000,436.000000,647.000000,6445.000000
population,15580.000000,1441.193068,1160.542775,3.000000,801.000000,1179.000000,1746.000000,35682.000000
households,15580.000000,501.099487,385.039835,2.000000,283.000000,411.000000,605.000000,6082.000000
median_income,15580.000000,3.672427,1.570297,0.499900,2.519975,3.440500,4.581425,15.000100
ocean_proximity_INLAND,15580.000000,0.333569,0.471503,0.000000,0.000000,0.000000,1.000000,1.000000
ocean_proximity_ISLAND,15580.000000,0.000193,0.013876,0.000000,0.000000,0.000000,0.000000,1.000000


# y =                                                              ETYKIETY (wartości)

In [20]:
train_labels = train_dataset.pop('median_house_value')   # ⏳
test_labels = test_dataset.pop('median_house_value')     # ⏳

# Standaryzacja

In [21]:
# funkcja do normalizacji
def norm(x):
    return (x - train_stats['mean']) / train_stats['std']

# Normalizacja

In [22]:
normed_train_data = norm(train_dataset)
normed_test_data = norm(test_dataset)

In [23]:
normed_train_data.isnull().sum()

longitude                     0
latitude                      0
housing_median_age            0
total_rooms                   0
total_bedrooms                0
population                    0
households                    0
median_income                 0
ocean_proximity_INLAND        0
ocean_proximity_ISLAND        0
ocean_proximity_NEAR BAY      0
ocean_proximity_NEAR OCEAN    0
dtype: int64

In [24]:
normed_train_data.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,ocean_proximity_INLAND,ocean_proximity_ISLAND,ocean_proximity_NEAR BAY,ocean_proximity_NEAR OCEAN
19234,-1.560743,1.332727,-0.838742,0.338845,-0.091639,0.000695,0.012727,1.918729,-0.707459,-0.013877,-0.346133,-0.378230
1859,-2.283490,2.934950,-0.598855,0.034607,0.089751,-0.200934,-0.018438,-0.889913,-0.707459,-0.013877,-0.346133,2.643727
51,-1.351396,1.011351,1.160315,-0.342501,-0.197646,-0.327599,-0.244389,-1.380585,-0.707459,-0.013877,2.888874,-0.378230
11192,0.811858,-0.851700,-0.039119,-0.080165,0.035570,0.350532,0.171152,-0.416818,-0.707459,-0.013877,-0.346133,-0.378230
20355,0.298459,-0.679368,-0.998666,-0.370283,-0.456775,-0.736029,-0.532671,-1.077075,-0.707459,-0.013877,-0.346133,-0.378230


In [25]:
normed_test_data.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,ocean_proximity_INLAND,ocean_proximity_ISLAND,ocean_proximity_NEAR BAY,ocean_proximity_NEAR OCEAN
10,-1.346412,1.025324,1.879976,-0.190383,-0.249472,-0.457711,-0.257375,-0.298878,-0.707459,-0.013877,2.888874,-0.378230
13,-1.346412,1.020666,1.879976,-0.876283,-0.821910,-0.944552,-0.849521,-0.636075,-0.707459,-0.013877,2.888874,-0.378230
19,-1.351396,1.020666,1.879976,-0.508739,-0.569849,-0.647277,-0.587211,-0.680844,-0.707459,-0.013877,2.888874,-0.378230
26,-1.356380,1.025324,1.640089,-0.678620,-0.697057,-0.718796,-0.680707,-0.772292,-0.707459,-0.013877,2.888874,-0.378230
40,-1.346412,1.016008,1.879976,-0.434957,-0.284807,-0.426691,-0.275555,-1.002758,-0.707459,-0.013877,2.888874,-0.378230


# Tworzymy obiekty numpy array

In [26]:
normed_test_data = normed_test_data.values     # ⏳
normed_train_data = normed_train_data.values   # ⏳

# Budowa modelu

In [27]:
def build_model():
  model = Sequential()
  model.add(Dense(units=1024, kernel_regularizer='l2', activation='relu', input_shape=[len(train_dataset.keys())]))
  model.add(Dense(512))
  model.add(Dense(128))
  model.add(Dense(1))  # 👈 regresja więc brak funkcji aktywacji domyślna jest linear


  model.compile(optimizer='adam',
                loss='mse',
                metrics=['mae', 'mse'])

  return model

In [28]:
model = build_model()
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 1024)              13312     
                                                                 
 dense_1 (Dense)             (None, 512)               524800    
                                                                 
 dense_2 (Dense)             (None, 128)               65664     
                                                                 
 dense_3 (Dense)             (None, 1)                 129       
                                                                 
Total params: 603905 (2.30 MB)
Trainable params: 603905 (2.30 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


# Trenowanie sieci

In [29]:
history = model.fit(normed_train_data, train_labels.values, epochs=150, validation_split=0.2, batch_size=32, verbose=1)

Epoch 1/150
390/390 [==============================] - 7s 16ms/step - loss: 14511162368.0000 - mae: 85117.7422 - mse: 14511162368.0000 - val_loss: 4189640704.0000 - val_mae: 47342.6719 - val_mse: 4189640704.0000
Epoch 2/150
390/390 [==============================] - 7s 18ms/step - loss: 3938068480.0000 - mae: 45069.6328 - mse: 3938068480.0000 - val_loss: 3640665344.0000 - val_mae: 44053.0781 - val_mse: 3640665344.0000
Epoch 3/150
390/390 [==============================] - 6s 16ms/step - loss: 3655421440.0000 - mae: 43371.5938 - mse: 3655421440.0000 - val_loss: 3491583744.0000 - val_mae: 43250.8516 - val_mse: 3491583744.0000
Epoch 4/150
390/390 [==============================] - 7s 19ms/step - loss: 3543484928.0000 - mae: 42633.1875 - mse: 3543484928.0000 - val_loss: 3400812544.0000 - val_mae: 42056.8906 - val_mse: 3400812544.0000
Epoch 5/150
390/390 [==============================] - 6s 15ms/step - loss: 3491598848.0000 - mae: 42189.8398 - mse: 3491598848.0000 - val_loss: 3376648192.00

In [30]:
def plot_hist(history):
    hist = pd.DataFrame(history.history)
    hist['epoch'] = history.epoch
    hist['rmse'] = np.sqrt(hist['mse'])
    hist['val_rmse'] = np.sqrt(hist['val_mse'])

    fig = go.Figure()
    fig.add_trace(go.Scatter(x=hist['epoch'], y=hist['mae'], name='mae', mode='markers+lines'))
    fig.add_trace(go.Scatter(x=hist['epoch'], y=hist['val_mae'], name='val_mae', mode='markers+lines'))
    fig.update_layout(width=1000, height=500, title='MAE vs. VAL_MAE', xaxis_title='Epoki', yaxis_title='Mean Absolute Error', yaxis_type='log')
    fig.show()

    fig = go.Figure()
    fig.add_trace(go.Scatter(x=hist['epoch'], y=hist['rmse'], name='rmse', mode='markers+lines'))
    fig.add_trace(go.Scatter(x=hist['epoch'], y=hist['val_rmse'], name='val_rmse', mode='markers+lines'))
    fig.update_layout(width=1000, height=500, title='RMSE vs. VAL_RMSE', xaxis_title='Epoki', yaxis_title='Root Mean Squared Error', yaxis_type='log')
    fig.show()

plot_hist(history)

In [31]:
for name, value in zip(model.metrics_names, model.evaluate(normed_test_data, test_labels.values)):
    print(f'{name:8}{value:.4f}')

122/122 [==============================] - 1s 6ms/step - loss: 2420085504.0000 - mae: 33833.8125 - mse: 2420085504.0000
loss    2420085504.0000
mae     33833.8125
mse     2420085504.0000


val_mae close to mae 👈👌

In [32]:
test_predictions = model.predict(normed_test_data).flatten()
test_predictions

122/122 [==============================] - 0s 3ms/step


array([226881.97 , 191033.58 , 172744.23 , ..., 111659.055, 146640.66 ,  91316.09 ], dtype=float32)

In [33]:
pred = pd.DataFrame(test_labels)
pred['predictions'] = test_predictions
pred.head()

,median_house_value,predictions
10,281500.000000,226881.968750
13,191300.000000,191033.578125
19,162900.000000,172744.234375
26,93800.000000,146679.953125
40,155400.000000,173155.750000


In [36]:
fig = px.scatter(pred, 'median_house_value', 'predictions')
fig.add_trace(go.Scatter(x=[0, 500000], y=[0, 500000], mode='lines'))
fig.show()

In [37]:
pred.head()  # predykcje konta wartości prawdziwe

,median_house_value,predictions
10,281500.000000,226881.968750
13,191300.000000,191033.578125
19,162900.000000,172744.234375
26,93800.000000,146679.953125
40,155400.000000,173155.750000


In [38]:
pred['error'] = pred['median_house_value'] - pred['predictions']
pred.head()

,median_house_value,predictions,error
10,281500.000000,226881.968750,54618.031250
13,191300.000000,191033.578125,266.421875
19,162900.000000,172744.234375,-9844.234375
26,93800.000000,146679.953125,-52879.953125
40,155400.000000,173155.750000,-17755.750000


In [39]:
px.histogram(pred, 'error', marginal='rug', width=1000)